In [1]:
!pwd

/home/jupyter/siim


In [2]:
%cp /home/jhunjhunwala_uday/train.csv .

In [3]:
!ls

exp1.ipynb  kaggle.json  train.csv


In [1]:
!pip install pytorch_lightning=='1.0.0rc5' > /dev/null
!pip install albumentations > /dev/null
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from PIL import Image
from sklearn.model_selection import StratifiedKFold
import pytorch_lightning as pl
np.random.seed(10)
torch.manual_seed(300)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

In [2]:
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.transforms as transforms
from fastai.vision import *
import PIL
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from pytorch_lightning.loggers import TensorBoardLogger
import albumentations
from torch.optim.lr_scheduler import OneCycleLR

In [3]:
torch.cuda.device(0)
use_cuda=torch.cuda.is_available()
device=torch.device('cuda:0' if use_cuda else 'cpu')

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.version.cuda

'10.1'

In [8]:
!sudo mv "/home/jhunjhunwala_uday/siic-isic-224x224-images.zip" .

In [6]:
MAIN_PATH='siim-isic-melanoma-classification/'
train_file='train.csv'
train_images='/home/jhunjhunwala_uday/train/'
train_images_384='jpeg-melanoma-384x384/train/'
train_images_512='jpeg-melanoma-512x512/train/'

In [7]:
train_data=pd.read_csv(train_file)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0


In [8]:
class_m=train_data['target'].sum()
class_b=train_data['target'].count()-class_m
class_weights=torch.tensor([1.0/class_b,1.0/class_m]).float().to(device)
class_weights

tensor([3.0730e-05, 1.7123e-03], device='cuda:0')

In [9]:
dummies=pd.get_dummies(train_data['anatom_site_general_challenge'],dummy_na=True,dtype=np.uint8, prefix='site')
dummies[:3]

,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0


In [10]:
train_data=pd.concat([train_data,dummies.iloc[:train_data.shape[0]]],axis=1)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [11]:
train_data['sex']=train_data['sex'].map({'male':1,'female':0})
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [12]:
train_data['sex']=train_data['sex'].fillna(-1)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [13]:
train_data['age_approx']/=90
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,0.555556,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [14]:
train_data['age_approx']=train_data['age_approx'].fillna(0)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,0.555556,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [15]:
meta_features=['sex','age_approx']+[col for col in train_data.columns if col.startswith('site_')]
n_meta_features=len(meta_features)
meta_features

['sex',
 'age_approx',
 'site_head/neck',
 'site_lower extremity',
 'site_oral/genital',
 'site_palms/soles',
 'site_torso',
 'site_upper extremity',
 'site_nan']

In [16]:
image_size=224
train_transforms=albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightness(limit=0.2, p=0.75),
    albumentations.RandomContrast(limit=0.2, p=0.75),
    albumentations.OneOf([
        albumentations.MotionBlur(blur_limit=5),
        albumentations.MedianBlur(blur_limit=5),
        albumentations.GaussianBlur(blur_limit=5),
        albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

    albumentations.OneOf([
        albumentations.OpticalDistortion(distort_limit=1.0),
        albumentations.GridDistortion(num_steps=5, distort_limit=1.),
        albumentations.ElasticTransform(alpha=3),
        ], p=0.7),
    albumentations.CLAHE(clip_limit=4.0, p=0.7),
    albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    albumentations.Cutout(max_h_size=int( image_size* 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
    albumentations.Normalize()
    
])

valid_transforms=albumentations.Compose([
    albumentations.Normalize()
    
])


In [17]:
class MelanomaDataset(Dataset):
    def __init__(self,df,idxs,transforms):
        if idxs is None:
            self.df=df
            return
        self.df=df.loc[idxs].reset_index(drop=True)
        self.transforms=transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        #pil2tensor=transforms.ToTensor()
        #img=pil2tensor(Image.open(train_images+self.df.loc[idx,'image_name']+'.png'))
        img=PIL.Image.open(train_images+self.df.loc[idx,'image_name']+'.png')
        img=np.array(img)
        img=self.transforms(image=img)['image']
        img=img.transpose(2,0,1)
        img=torch.tensor(img).float()
        meta_data=torch.tensor(self.df.iloc[idx][meta_features]).float()
        return ((img,meta_data),self.df.loc[idx,'target'])
    def get_targets(self):
        targets=list(self.df['target'])
        return targets

In [18]:
y=train_data['target'].to_numpy()
x=np.zeros(33126)
skf=StratifiedKFold(n_splits=5)
train=[]
valid=[]
counter=0
for train_idx,test_idx in skf.split(x,y):
    train.append(train_idx)
    valid.append(test_idx)
    counter+=1
    total=0
    for i in test_idx:
        total+=y[i]
    print(total)

117
116
117
117
117


In [19]:
train_dataset=MelanomaDataset(train_data,train[0],train_transforms)
valid_dataset=MelanomaDataset(train_data,valid[0],valid_transforms)
print(len(train_dataset))
print(len(valid_dataset))

26500
6626


In [20]:
label_weights=[class_weights[x] for x in train_dataset.get_targets()]
print(len(label_weights))
#print(label_weights)

26500


In [21]:
total_dataset_size=26500
sampler = WeightedRandomSampler(label_weights,total_dataset_size)

In [22]:
train_dataloader=torch.utils.data.DataLoader(train_dataset,shuffle=False,batch_size=256,sampler=sampler,
                                             pin_memory=True,num_workers=4)

In [23]:
valid_dataloader=DataLoader(valid_dataset,shuffle=False,batch_size=256,pin_memory=True,num_workers=4,drop_last=True)

In [107]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

class MelanomaModel(nn.Module):
    def __init__(self):
        super().__init__()
        num_target_classes=2
        self.classifier = EfficientNet.from_pretrained('efficientnet-b2')
        self.bn1=nn.BatchNorm2d(1408,eps=1e-3,momentum=0.01)
        self.adaptive=nn.AdaptiveAvgPool2d(output_size=1)
        #self.classifier.eval()
        #self.classifier.avgpool=AdaptiveConcatPool2d(output_size=1)
        '''actual_layer1=nn.Sequential(
                                self.classifier.conv1,
                                self.classifier.bn1,
                                self.classifier.relu,
                                self.classifier.maxpool,
                                self.classifier.layer1,
                                self.classifier.layer2,
                                self.classifier.layer3,
                                self.classifier.layer4,
                                self.classifier.avgpool
                            )
      
        fc1=nn.Sequential(
                            nn.Flatten(),
                            nn.BatchNorm1d(1024),
                            nn.Dropout(p=0.25),
                            nn.Linear(1024,512),
                            nn.ReLU(),
                            nn.BatchNorm1d(512))
        fc=nn.Sequential(
                            nn.Flatten(),
                            nn.BatchNorm1d(1024),
                            nn.Dropout(p=0.25),
                            nn.Linear(1024,512),
                            nn.ReLU(),
                            nn.BatchNorm1d(512),
                            nn.Dropout(p=0.5),
                            nn.Linear(512, num_target_classes))'''
        
        #self.classifier=nn.Sequential(self.classifier, big)
        self.meta=nn.Sequential(
                                nn.Linear(n_meta_features, 512),
                                nn.ReLU(),
                                nn.BatchNorm1d(512),
                                nn.Dropout(p=0.3),
                                nn.Linear(512,256),
                                nn.ReLU(),
                                nn.BatchNorm1d(256))
                                
        self.final_layer=nn.Sequential(
                                        nn.Linear(1664,256),
                                        nn.ReLU(),
                                        nn.BatchNorm1d(256),
                                        nn.Dropout(p=0.3),
                                        nn.Linear(256,2))
        
        #self.updateable_params=[]
        #for p in self.classifier.fc.parameters():
        #    self.updateable_params.append(p)
        # use the pretrained model to classify cifar-10 (10 image classes)
        
    def forward(self,x,x_meta):
        x=self.classifier.extract_features(x)
        x=self.bn1(x)
        x=self.adaptive(x)
        x=x.squeeze(-1).squeeze(-1)
        #print(x.shape)
        #x=nn.Flatten(x)
        #x1=self.classifier(x)
        x_meta=self.meta(x_meta)
        #print(x_meta.shape)
        x=torch.cat((x,x_meta),dim=1)
        return self.final_layer(x)

In [34]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16034 sha256=47c5de74cdea366433cabb833ab438f3444789d755d1a9e4c2e60f1c3e5a14df
  Stored in directory: /home/jupyter/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch


In [39]:
model=MelanomaModel()
model.to(device)

Loaded pretrained weights for efficientnet-b2


MelanomaModel(
  (classifier): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
     

In [40]:
data=DataBunch(train_dataloader, valid_dataloader,device=device)

In [41]:
learn=Learner(data,model,loss_func=F.cross_entropy, metrics=[error_rate,accuracy,AUROC()])

In [42]:
for p in learn.model.classifier.parameters():
    p.requires_grad_(False)

In [43]:
counter=0
total_params=0
for p in learn.model.parameters():
    total_params+=1
    if p.requires_grad:
        counter+=1
print(counter)
print(total_params)

16
317


In [44]:
learn.model.to(device)
learn.data.device

device(type='cuda', index=0)

In [45]:
counter=0
for p in learn.model.parameters():
    if p.requires_grad:
        counter+=1
print(counter)

16


In [46]:
learn.fit_one_cycle(2,max_lr=1e-3)

epoch,train_loss,valid_loss,error_rate,accuracy,auroc,time
0,0.572475,0.236053,0.066406,0.933594,0.801801,04:24
1,0.523997,0.363957,0.165000,0.835000,0.827466,04:22


In [24]:
pip install efficientnet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [24]:
from efficientnet_pytorch import EfficientNet

In [25]:
from pytorch_lightning.metrics.functional.classification import auroc as pl_auroc
from pytorch_lightning.metrics.functional.classification import accuracy as pl_accuracy
from pytorch_lightning.metrics.functional.classification import auc
#from pytorch_lightning.metrics.sklearns import AUROC as AUC

class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
class MelanomaModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        num_target_classes=2
        self.classifier = EfficientNet.from_pretrained('efficientnet-b2')
        self.bn1=nn.BatchNorm2d(1408,eps=1e-3,momentum=0.01)
        self.adaptive=nn.AdaptiveAvgPool2d(output_size=1)
        
        self.meta=nn.Sequential(
                                nn.Linear(n_meta_features, 512),
                                nn.ReLU(),
                                nn.BatchNorm1d(512),
                                nn.Dropout(p=0.3),
                                nn.Linear(512,256),
                                nn.ReLU(),
                                nn.BatchNorm1d(256))
                                
        self.final_layer=nn.Sequential(
                                        nn.Linear(1664,256),
                                        nn.ReLU(),
                                        nn.BatchNorm1d(256),
                                        nn.Dropout(p=0.3),
                                        nn.Linear(256,2))
        #self.auroc=AUROC()
        #self.auc=AUC()
        #self.accuracy=Accuracy(num_classes=2)
        
        self.updateable_params=[]
        self.lr_values=[]
        for p in self.final_layer.parameters():
            self.updateable_params.append(p)
        for p in self.meta.parameters():
            self.updateable_params.append(p)
        for p in self.bn1.parameters():
            self.updateable_params.append(p)
        self.total_params=list(self.updateable_params)
        for p in self.classifier.parameters():
            self.total_params.append(p)
        #self.updateable_params=[]
        #for p in self.classifier.fc.parameters():
        #    self.updateable_params.append(p)
        # use the pretrained model to classify cifar-10 (10 image classes)
        
    def forward(self,x1):
        x,x_meta=x1
        x=self.classifier.extract_features(x)
        x=self.bn1(x)
        x=self.adaptive(x)
        x=x.squeeze(-1).squeeze(-1)
        #print(x.shape)
        #x=nn.Flatten(x)
        #x1=self.classifier(x)
        x_meta=self.meta(x_meta)
        #print(x_meta.shape)
        x=torch.cat((x,x_meta),dim=1)
        return self.final_layer(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        accuracy=pl_accuracy(y_hat,y)
        #result=pl.TrainResult(loss,early_stop_on=loss)
        self.log_dict({'train_loss':loss,'train_accuracy':accuracy},
                         on_step=False,on_epoch=True,reduce_fx=torch.mean,prog_bar=True)
        self.log('train_loss',loss)
        self.log('train_accuracy',accuracy)
        return {'loss':loss,'accuracy':accuracy}
    
    def training_epoch_end(self,outputs):
        avg_loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().item()
        avg_accuracy=torch.stack([x['accuracy'] for x in outputs]).mean().detach().cpu().item()
        print('---')
        print('train_avg_loss:'+str(avg_loss))
        print('train_avg_accuracy:'+str(avg_accuracy))
        
        self.log('epoch_train_loss',avg_loss)
        self.log('epoch_train_accuracy',avg_accuracy)
        #return {'train_avg_loss':avg_loss,'train_avg_accuracy':avg_accuracy}
    
    '''def optimizer_step(self, current_epoch, batch_idx, optimizer, optimizer_idx,
                   second_order_closure, on_tpu, using_native_amp, using_lbfgs):
        optimizer.step()
        optimizer.zero_grad()
        self.lr_values.append(optimizer.param_groups[0]['lr'])
        self.scheduler.step()'''
        
    def on_train_batch_end(self,outputs, batch, batch_idx, dataloader_idx):
        #print('calling lr scheduler')
        #self.lr_values.append(self.optimizer.param_groups[0]['lr'])
        self.scheduler.step()
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        y_hat= self(x)
        loss = F.cross_entropy(y_hat,y)
        #auroc=self.auroc(y_hat,y)
        accuracy=pl_accuracy(y_hat,y)
        #result=pl.EvalResult(checkpoint_on=loss)
        self.log_dict({'val_loss':loss,'val_accuracy':accuracy},
                        on_step=False,on_epoch=True,reduce_fx=torch.mean,prog_bar=True)
        #result.loss=loss
        #result.y_hat=y_hat
        #result.y=y
        self.log('val_loss',loss)
        self.log('val_accuracy',accuracy)
        return {'loss':loss,'y_hat':y_hat,'y':y,'accuracy':accuracy}

       # return pl.EvalResult()
    
    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.updateable_params, lr=1e-3, weight_decay=0.01)
        self.optimizer=optimizer
        self.scheduler=OneCycleLR(optimizer,max_lr=1e-3, epochs=2, steps_per_epoch=len(train_dataloader))
        return optimizer
        #return [optimizer],[scheduler]
    
    def validation_epoch_end(self, outputs):
        
        #avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        #print(outputs)
        #print(outputs[0]['y'].shape)
        #print(outputs[0]['y_hat'].shape)
        avg_loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().item()
        avg_accuracy=torch.stack([x['accuracy'] for x in outputs]).mean().detach().cpu().item()
        
        all_y=torch.stack([x['y'] for x in outputs],dim=0)
        all_y_hat=torch.stack([x['y_hat'] for x in outputs],dim=0)
        all_y=all_y.view(-1)
        all_y_hat=all_y_hat.view(-1,2)
        #print(all_y.shape)
        #print(all_y_hat.shape)
        #print(len(outputs[0]))
        
        auroc=pl_auroc(all_y_hat[:,1],all_y)
        #copy_all_y=all_y.clone().int().cpu().numpy()
        #c_yhat=all_y_hat[:,1].clone().cpu().numpy()
        
        #auc=self.auc(c_yhat,copy_all_y)
        copy_auroc=auroc.clone()
        #print('auroc'+str(auroc.to('cpu').item()))
        print('----')
        print('auroc:'+str(copy_auroc.detach().cpu().item()))
        #print('auc:'+str(auc.detach().cpu().item()))
        print('avg_loss:'+str(avg_loss))
        print('avg_accuracy:'+str(avg_accuracy))
        self.log('epoch_auroc',auroc)
        return {'auroc':auroc}
        #return {"auroc":auroc}
    
    '''def setup(self,stage):
        img_dataset=MelanomaDataset(self.df)
        train_size=int(0.8*(len(img_dataset)))
        val_size=len(img_dataset)-train_size
        self.train_set, self.val_set=random_split(img_dataset, (train_size,val_size))
        self.batch_size=64
    
    def train_dataloader(self):
        label_weights=[class_weights[x] for _,x in self.train_set]
        sampler = torch.utils.data.sampler.WeightedRandomSampler(label_weights, self.batch_size)
        return DataLoader(self.train_set, batch_size=self.batch_size, sampler=sampler, shuffle=False)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)'''

In [26]:
model2=MelanomaModel()

Loaded pretrained weights for efficientnet-b2


In [27]:
from pytorch_lightning.callbacks.progress import ProgressBar
from tqdm import tqdm

class CustomProgressBar(ProgressBar):
    def init_train_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for training. """
        bar = tqdm(
            desc='Training',
            initial=self.train_batch_idx,
            position=0,
            disable=self.is_disabled,
            leave=True,
            dynamic_ncols=True,
            file=sys.stdout,
            smoothing=0,
        )
        return bar
    

In [30]:
import wandb
from pytorch_lightning.loggers import WandbLogger
wandb.login()
wandb.init()
display(wandb.jupyter.Run())
wandb_logger = WandbLogger(offline=True)

wandb: Currently logged in as: udayj (use `wandb login --relogin` to force relogin)


In [31]:
progress_bar_callback=CustomProgressBar()

logger=TensorBoardLogger('logs','melanoma_1')
trainer=pl.Trainer(gpus=1,max_epochs=2,
                   callbacks=[progress_bar_callback], precision=16,logger=wandb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [32]:
tot=0
for p in model2.classifier.parameters():
    p.requires_grad_(False)
    tot+=1
print(tot)

updateable=0
tot=0
for p in model2.total_params:
    if p.requires_grad:
        updateable+=1
    tot+=1
print(updateable)
print(tot)

301
16
317


In [33]:
trainer.fit(model2, train_dataloader, valid_dataloader)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.



  | Name        | Type              | Params
--------------------------------------------------
0 | classifier  | EfficientNet      | 9 M   
1 | bn1         | BatchNorm2d       | 2 K   
2 | adaptive    | AdaptiveAvgPool2d | 0     
3 | meta        | Sequential        | 137 K 
4 | final_layer | Sequential        | 427 K 


----
auroc:0.4752964973449707
avg_loss:0.72590172290802
avg_accuracy:0.169921875
Epoch 0:   0%|          | 0/129 [00:00<?, ?it/s] 

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


Epoch 0:  81%|████████  | 104/129 [04:24<01:03,  2.54s/it, loss=0.558, v_num=vw0i]

Epoch 0: 100%|██████████| 129/129 [04:37<00:00,  2.15s/it, loss=0.558, v_num=vw0i]----
auroc:0.7489064931869507
avg_loss:0.3415071666240692
avg_accuracy:0.8764062523841858
Epoch 0: 100%|██████████| 129/129 [04:38<00:00,  2.16s/it, loss=0.558, v_num=vw0i]---
train_avg_loss:0.6029478907585144
train_avg_accuracy:0.6848161816596985
Epoch 1:  81%|████████  | 104/129 [04:25<01:03,  2.55s/it, loss=0.519, v_num=vw0i]

Epoch 1: 100%|██████████| 129/129 [04:39<00:00,  2.16s/it, loss=0.519, v_num=vw0i]----
auroc:0.7691260576248169
avg_loss:0.3585774004459381
avg_accuracy:0.8457812070846558
Epoch 1: 100%|██████████| 129/129 [04:39<00:00,  2.16s/it, loss=0.519, v_num=vw0i]---
train_avg_loss:0.5312756299972534
train_avg_accuracy:0.729892909526825
Epoch 1: 100%|██████████| 129/129 [04:39<00:00,  2.17s/it, loss=0.519, v_num=vw0i]


1

In [32]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

tqdm==4.48.2
torchvision==0.7.0+cu101
torch==1.6.0+cu101
scipy==1.5.2
scikit-learn==0.23.2
requests==2.24.0
Pillow==7.2.0
pandas==1.1.2
numpy==1.18.5
matplotlib==3.3.1
fastprogress==1.0.0
fastai==1.0.61
albumentations==0.4.6


In [1]:
!pip show pytorch_lightning

Name: pytorch-lightning
Version: 1.0.0rc5
Summary: PyTorch Lightning is the lightweight PyTorch wrapper for ML researchers. Scale your models. Write less boilerplate.
Home-page: https://github.com/PyTorchLightning/pytorch-lightning
Author: William Falcon et al.
Author-email: waf2107@columbia.edu
License: Apache-2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: fsspec, tqdm, torch, tensorboard, future, numpy, PyYAML
Required-by: 


In [41]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [42]:
torch.backends.cudnn.enabled

True

In [34]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [40]:
!pip install torch torchvision

In [48]:
%reload_ext tensorboard

In [52]:
%tensorboard --logdir logs --bind_all

Reusing TensorBoard on port 6006 (pid 23471), started 0:00:41 ago. (Use '!kill 23471' to kill it.)

In [50]:
!tensorboard --logdir /tmp/whatever

2020-10-16 10:07:11.981227: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:6008/ (Press CTRL+C to quit)
^C


In [29]:
!wandb login 139bdd758a7630ed8432a19ebd0bf7d05a3fe831

wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [28]:
import wandb